In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "../data/online_retail_II.xlsx"

df = pd.read_excel(file_path, sheet_name='Year 2010-2011')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.info()
df.describe(include='all')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data Cleaning

In [4]:
# removing rows with missing Customer ID
df = df.dropna(subset=['Customer ID'])

# removing negative quantities
df = df[df['Quantity'] > 0]

# removing zero or negative prices
df = df[df['Price'] > 0]

# creating TotalProce column
df['TotalPrice'] = df['Quantity'] * df['Price']

# converting InvoiceDate to DateTime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# renaming columns
df = df.rename(columns={
    'Customer ID': 'CustomerID',
    'StockCode': 'ProductCode',
    'Description': 'ProductDescription'
})

# removing blank descs
df = df[df['ProductDescription'].notna()]

#resetting index
df.reset_index(drop=True, inplace=True)

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397885 entries, 0 to 397884
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Invoice             397885 non-null  object        
 1   ProductCode         397885 non-null  object        
 2   ProductDescription  397885 non-null  object        
 3   Quantity            397885 non-null  int64         
 4   InvoiceDate         397885 non-null  datetime64[ns]
 5   Price               397885 non-null  float64       
 6   CustomerID          397885 non-null  float64       
 7   Country             397885 non-null  object        
 8   TotalPrice          397885 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 27.3+ MB


,Invoice,ProductCode,ProductDescription,Quantity,InvoiceDate,Price,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [25]:
# saving cleaned df to csv for power bi
df.to_csv("../data/retail_clean.csv", index=False)

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules

## Market Basket Analysis (Apriori + Association Rules)

Preparing data for basket analysis

In [28]:
# filtering columns 
basket_df = df[['Invoice', 'ProductDescription', 'Quantity']]

In [30]:
# grouping by events and product
basket = (basket_df
          .groupby(['Invoice', 'ProductDescription'])['Quantity']
          .sum()
          .unstack()
          .fillna(0))

In [32]:
# flagging quantities
def encode_units(x):
    return 1 if x >= 1 else 0

basket = basket.applymap(encode_units)

/var/folders/b7/6xhllfdx0m7fgcyfhk3_mnbr0000gn/T/ipykernel_23559/997237897.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(encode_units)


In [36]:
# generating frequent itemsets
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values('support', ascending=False).head()

/opt/anaconda3/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
192,0.106357,(WHITE HANGING HEART T-LIGHT HOLDER)
154,0.091895,(REGENCY CAKESTAND 3 TIER)
81,0.086337,(JUMBO BAG RED RETROSPOT)
124,0.074412,(PARTY BUNTING)
11,0.074196,(ASSORTED COLOUR BIRD ORNAMENT)


In [38]:
# generarting association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
74,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER , GREEN REGEN...",0.030002,0.029193,0.021045,0.701439,24.027846,1.0,0.020169,3.251619,0.988024,0.551627,0.692461,0.711163
71,"(ROSES REGENCY TEACUP AND SAUCER , GREEN REGEN...",(PINK REGENCY TEACUP AND SAUCER),0.029193,0.030002,0.021045,0.720887,24.027846,1.0,0.020169,3.475290,0.987201,0.551627,0.712254,0.711163
75,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER , PINK REGENC...",0.037287,0.023527,0.021045,0.564399,23.989564,1.0,0.020167,2.241671,0.995432,0.529172,0.553904,0.729447
70,"(ROSES REGENCY TEACUP AND SAUCER , PINK REGENC...",(GREEN REGENCY TEACUP AND SAUCER),0.023527,0.037287,0.021045,0.894495,23.989564,1.0,0.020167,9.124846,0.981405,0.529172,0.890409,0.729447
8,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.030002,0.037287,0.024822,0.827338,22.188466,1.0,0.023703,5.575714,0.984468,0.584498,0.820651,0.746520
9,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.037287,0.030002,0.024822,0.665702,22.188466,1.0,0.023703,2.901595,0.991917,0.584498,0.655362,0.746520
73,(ROSES REGENCY TEACUP AND SAUCER ),"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",0.042251,0.024822,0.021045,0.498084,20.066300,1.0,0.019996,1.942912,0.992082,0.457210,0.485309,0.672955
72,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",(ROSES REGENCY TEACUP AND SAUCER ),0.024822,0.042251,0.021045,0.847826,20.066300,1.0,0.019996,6.293778,0.974350,0.457210,0.841113,0.672955
62,(ROSES REGENCY TEACUP AND SAUCER ),(PINK REGENCY TEACUP AND SAUCER),0.042251,0.030002,0.023527,0.556833,18.559754,1.0,0.022259,2.188785,0.987858,0.482835,0.543125,0.670503
63,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.030002,0.042251,0.023527,0.784173,18.559754,1.0,0.022259,4.437569,0.975384,0.482835,0.774651,0.670503


In [40]:
# saving rules for power bi
rules.to_csv("../data/association_rules.csv", index=False)